In [8]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

관측소 코드          
백령도 47102   
오산 47122

In [7]:
# 관측소 코드 담기

where = 47122   # 오산

In [2]:
where = 47102   # 백령도

In [13]:
final = pd.DataFrame()

# 30일, 31일 월 분리
# 2월은 따로 생각해야 함
months_30 = [4, 6, 9, 11]
months_31 = [1, 3, 5, 7, 8, 10, 12]

for year in range(2014, 2022):   # 여기 바꾸기
    for month in range(1, 13): 
        if month in months_30: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3012&STNM={}'.format(year, month, where), verify=False)
        elif month in months_31: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3112&STNM={}'.format(year, month, where), verify=False)
        else:   # 2월인 경우
            if year % 4 == 0:   # 윤년인 경우 따로 분리 - 29일까지 있음
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2912&STNM={}'.format(year, month, where), verify=False)
            else:   # 28일까지 있는 경우
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2812&STNM={}'.format(year, month, where), verify=False)
        
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        title = soup.select('body > h2')  # title은 하나씩 있음
        data = soup.select('body > pre')  # pre는 하나 건너 하나씩 concat 해야함

        idx_data = 0
        for idx in range(len(title)): 
            data_df = pd.DataFrame(data[idx_data])
            data_df[0] = data_df[0].str.replace("-----------------------------------------------------------------------------", "")
            data_df_2 = data_df[0].str.split('\n', expand=True)

            data_df_2.drop([0, 1, 2, 3, 4], axis = 1, inplace = True)

            data_df_2 = data_df_2.transpose()
            title_tmp = str(title[idx])
            data_df_2['date'] = title_tmp[4:54]   # 백령도 기준 4:56, 오산 기준 4:54
            
            final = pd.concat([final, data_df_2], axis = 0)

            idx_data += 2

C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\urllib3\connecti

In [ ]:
# 오산 14 ~ 21 돌리는 중

In [9]:
# 오산 2018-11, 2018-12 추가

final = pd.DataFrame()

# 30일, 31일 월 분리
# 2월은 따로 생각해야 함
months_30 = [4, 6, 9, 11]
months_31 = [1, 3, 5, 7, 8, 10, 12]

for year in range(2018, 2019):   # 여기 바꾸기
    for month in range(11, 13): 
        if month in months_30: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3012&STNM={}'.format(year, month, where), verify=False)
        elif month in months_31: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3112&STNM={}'.format(year, month, where), verify=False)
        else:   # 2월인 경우
            if year % 4 == 0:   # 윤년인 경우 따로 분리 - 29일까지 있음
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2912&STNM={}'.format(year, month, where), verify=False)
            else:   # 28일까지 있는 경우
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2812&STNM={}'.format(year, month, where), verify=False)
        
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        title = soup.select('body > h2')  # title은 하나씩 있음
        data = soup.select('body > pre')  # pre는 하나 건너 하나씩 concat 해야함

        idx_data = 0
        for idx in range(len(title)): 
            data_df = pd.DataFrame(data[idx_data])
            data_df[0] = data_df[0].str.replace("-----------------------------------------------------------------------------", "")
            data_df_2 = data_df[0].str.split('\n', expand=True)

            data_df_2.drop([0, 1, 2, 3, 4], axis = 1, inplace = True)

            data_df_2 = data_df_2.transpose()
            title_tmp = str(title[idx])
            data_df_2['date'] = title_tmp[4:54]   # 백령도 기준 4:56, 오산 기준 4:54
            
            final = pd.concat([final, data_df_2], axis = 0)

            idx_data += 2

C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [10]:
title_tmp[4:54]

'47122 RKSO Osan Ab Observations at 12Z 31 Dec 2018'

In [11]:
final

,0,date
5,1021.0 52 3.2 0.8 84 3.99 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
6,1016.0 92 2.2 -2.4 72 3.17 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
7,1000.0 222 4.8 -1.2 65 3.52 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
8,999.0 230 4.8 -1.2 65 3.52 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
9,981.0 378 5.2 -2.8 56 3.18 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
...,...,...
83,10.0 30540 -52.5 -84.5 1 0.03 ...,47122 RKSO Osan Ab Observations at 12Z 31 Dec ...
84,9.0 31225 -51.4 -83.7 1 0.04 ...,47122 RKSO Osan Ab Observations at 12Z 31 Dec ...
85,8.0 31991 -50.1 -82.9 1 0.05 ...,47122 RKSO Osan Ab Observations at 12Z 31 Dec ...
86,7.6 32325 -49.5 -82.5 1 0.06 ...,47122 RKSO Osan Ab Observations at 12Z 31 Dec ...


In [12]:
final2 = final.reset_index(drop = True)
final2

,0,date
0,1021.0 52 3.2 0.8 84 3.99 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
1,1016.0 92 2.2 -2.4 72 3.17 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
2,1000.0 222 4.8 -1.2 65 3.52 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
3,999.0 230 4.8 -1.2 65 3.52 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
4,981.0 378 5.2 -2.8 56 3.18 ...,47122 RKSO Osan Ab Observations at 00Z 01 Nov ...
...,...,...
20325,10.0 30540 -52.5 -84.5 1 0.03 ...,47122 RKSO Osan Ab Observations at 12Z 31 Dec ...
20326,9.0 31225 -51.4 -83.7 1 0.04 ...,47122 RKSO Osan Ab Observations at 12Z 31 Dec ...
20327,8.0 31991 -50.1 -82.9 1 0.05 ...,47122 RKSO Osan Ab Observations at 12Z 31 Dec ...
20328,7.6 32325 -49.5 -82.5 1 0.06 ...,47122 RKSO Osan Ab Observations at 12Z 31 Dec ...


In [13]:
final2.to_csv('D:\\SNUlab\\Osan_crawling_1811-1812.csv', encoding = 'euc-kr', index = False)

In [244]:
data_df = pd.DataFrame(data[0])
data_df[0] = data_df[0].str.replace("-----------------------------------------------------------------------------", "")
data_df_2 = data_df[0].str.split('\n', expand=True)
# data_df_2 = data_df_2.str.replace("  ", "")
# countcol = len(data_df_2.columns) - 13

# datalist = []
# datalist.append(data_df_2.values)
# datalist = np.array(datalist)

# datalist.reshape(countcol // 11, 11)
# reshaped = data_df_2.reshape(countcol / 10, 10)
data_df_2.drop([0, 1, 2, 3, 4], axis = 1, inplace = True)

# df = pd.DataFrame({'PRES' : [], 'HGHT' : [], 'TEMP' : [], 'DWPT' : [], 'RELH' : [], 'MIXR' : [], 'DRCT': [], 'SKNT': [], 'THTA' : [], 'THTE': [], 'THTV': []})
# for idx in range(5, len(data_df_2.columns)): 
#     row = data_df_2[[idx][0]].str.replace("   ", ";")
#     row = row.str.replace("  ", ";")
#     row = row.str.replace(" ", ";")
#     row = row.str.replace(";;", ";")
#     print(row)
#     row = row.str.split(";", expand = True)
#     df = pd.concat([df, row], axis = 0)

data_df_2 = data_df_2.transpose()
title_tmp = str(title[0])
data_df_2['date'] = title_tmp[4:56]
data_df_2

,0,date
5,1012.0 158 -3.9 -9.9 63 1.79 ...,47102 Baengnyeongdo Observations at 00Z 01 De...
6,1000.0 254 -4.9 -10.9 63 1.67 ...,47102 Baengnyeongdo Observations at 00Z 01 De...
7,991.0 325 -5.5 -11.3 64 1.64 ...,47102 Baengnyeongdo Observations at 00Z 01 De...
8,946.0 687 -8.7 -13.1 71 1.48 ...,47102 Baengnyeongdo Observations at 00Z 01 De...
9,941.0 728 -9.1 -13.3 72 1.46 ...,47102 Baengnyeongdo Observations at 00Z 01 De...
...,...,...
111,6.0 34319 -43.7 -78.7 1 0.14 ...,47102 Baengnyeongdo Observations at 00Z 01 De...
112,5.0 35561 -37.3 -74.3 1 0.32 ...,47102 Baengnyeongdo Observations at 00Z 01 De...
113,4.2 36770 -35.3 -72.3 1 0.52 ...,47102 Baengnyeongdo Observations at 00Z 01 De...
114,4.0 ...,47102 Baengnyeongdo Observations at 00Z 01 De...


In [243]:
title_tmp[4:56]

'47102  Baengnyeongdo Observations at 00Z 01 Dec 2012'

In [245]:
data_df[1]

KeyError: 1

In [147]:
df = pd.DataFrame({'PRES' : [], 'HGHT' : [], 'TEMP' : [], 'DWPT' : [], 'RELH' : [], 'MIXR' : [], 'DRCT': [], 'SKNT': [], 'THTA' : [], 'THTE': [], 'THTV': []})
df

,PRES,HGHT,TEMP,DWPT,RELH,MIXR,DRCT,SKNT,THTA,THTE,THTV


In [181]:
data_df_2

,5,6,7,8,9,10,11,12,13,14,...,18,19,20,21,22,23,24,25,26,27
0,1024.0 52 -5.9 -11.9 63 1.51 ...,1000.0 234 -6.1 -12.1 63 1.52 ...,979.0 400 -7.3 -12.3 67 1.53 ...,942.0 699 -8.7 -15.7 57 1.20 ...,925.0 840 -8.5 -20.5 37 0.81 ...,908.0 984 -7.3 -34.3 10 0.23 ...,853.0 1469 -9.4 -36.4 9 0.20 ...,850.0 1496 -9.5 -36.5 9 0.20 ...,836.0 1624 -10.1 -37.1 9 0.19 ...,736.0 2606 -9.7 -38.1 8 0.20 ...,...,500.0 5550 -21.3 -38.3 20 0.28 ...,491.0 5684 -22.5 -36.5 27 0.34 ...,406.0 7055 -31.7 -43.7 30 0.20 ...,400.0 7160 -32.7 -44.7 29 0.18 ...,399.0 7178 -32.9 -44.8 29 0.18 ...,383.0 7464 -35.5 -45.5 35 0.17 ...,372.0 7666 -36.9 -47.5 33 0.14 ...,364.0 7816 -37.9 -48.9 31 0.12 ...,363.0 7835 -36.9 -47.9 31 0.14 ...,


In [184]:
data_df_2[[17]]

,17
0,606.0 4099 -11.5 -38.3 9 0.23 ...
